In [19]:
import pandas as pd
import pickle

import seaborn as sns
import matplotlib.pyplot as plt

In [20]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [21]:
import mlflow

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1720947356054, experiment_id='1', last_update_time=1720947356054, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [22]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [23]:
df_train = read_dataframe('./data/green_tripdata_2024-01.parquet')
df_val = read_dataframe('./data/green_tripdata_2024-02.parquet')

In [24]:
len(df_train), len(df_val)

(54373, 51497)

In [25]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [26]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [27]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [28]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

5.982610254412603

In [29]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [30]:
with mlflow.start_run():

    mlflow.set_tag("developer", "tejash")

    mlflow.log_param("train-data-path", "./data/green_tripdata_2024-01.parquet")
    mlflow.log_param("valid-data-path", "./data/green_tripdata_2024-02.parquet")

    alpha = 0.1
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")

MlflowException: The configured tracking uri scheme: 'sqlite' is invalid for use with the proxy mlflow-artifact scheme. The allowed tracking schemes are: {'http', 'https'}

In [6]:
!pip install xgboost

     |████████████████████████████████| 153.9 MB 54 kB/s s eta 0:00:01     |████████████████████████▌       | 118.0 MB 55.3 MB/s eta 0:00:01
     |████████████████████████████████| 190.9 MB 21 kB/s s eta 0:00:01    |███████████                     | 65.4 MB 99.4 MB/s eta 0:00:02


In [11]:
import xgboost as xgb

In [12]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [13]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [19]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [20]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

  0%|                                                                                                              | 0/50 [00:00<?, ?trial/s, best loss=?]

/home/codespace/anaconda3/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [20:37:44] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:8.62354                                                                                                                               
[1]	validation-rmse:8.19155                                                                                                                               
[2]	validation-rmse:7.81474                                                                                                                               
[3]	validation-rmse:7.47692                                                                                                                               
[4]	validation-rmse:7.18436                                                                                                                               
[5]	validation-rmse:6.93117                                                                                                                               
[6]	validation-rmse:6.70976                                           

[104]	validation-rmse:5.26660                                                                                                                             
[105]	validation-rmse:5.26689                                                                                                                             
[106]	validation-rmse:5.26673                                                                                                                             
[107]	validation-rmse:5.26666                                                                                                                             
[108]	validation-rmse:5.26654                                                                                                                             
[109]	validation-rmse:5.26669                                                                                                                             
[110]	validation-rmse:5.26680                                         

[208]	validation-rmse:5.25663                                                                                                                             
[209]	validation-rmse:5.25667                                                                                                                             
[210]	validation-rmse:5.25655                                                                                                                             
[211]	validation-rmse:5.25648                                                                                                                             
[212]	validation-rmse:5.25669                                                                                                                             
[213]	validation-rmse:5.25707                                                                                                                             
[214]	validation-rmse:5.25719                                         

...

[111]	validation-rmse:5.43209                                                                                                                             
[112]	validation-rmse:5.43041                                                                                                                             
[113]	validation-rmse:5.43027                                                                                                                             
[114]	validation-rmse:5.42865                                                                                                                             
[115]	validation-rmse:5.42800                                                                                                                             
[116]	validation-rmse:5.42690                                                                                                                             
[117]	validation-rmse:5.42643                                         

[215]	validation-rmse:5.37699                                                                                                                             
[216]	validation-rmse:5.37633                                                                                                                             
[217]	validation-rmse:5.37587                                                                                                                             
[218]	validation-rmse:5.37405                                                                                                                             
[219]	validation-rmse:5.37462                                                                                                                             
[220]	validation-rmse:5.37434                                                                                                                             
[221]	validation-rmse:5.37563                                         

/home/codespace/anaconda3/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [21:09:29] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:6.46830                                                                                                                               
[1]	validation-rmse:5.64405                                                                                                                               
[2]	validation-rmse:5.40790                                                                                                                               
[3]	validation-rmse:5.30724                                                                                                                               
[4]	validation-rmse:5.27795                                                                                                                               
[5]	validation-rmse:5.26793                                                                                                                               
[6]	validation-rmse:5.26348                                           

[104]	validation-rmse:5.16265                                                                                                                             
[105]	validation-rmse:5.16203                                                                                                                             
[106]	validation-rmse:5.16192                                                                                                                             
[107]	validation-rmse:5.16256                                                                                                                             
[108]	validation-rmse:5.16074                                                                                                                             
[109]	validation-rmse:5.16152                                                                                                                             
[110]	validation-rmse:5.16258                                         

In [15]:
with mlflow.start_run():
    
    best_params = {
        'learning_rate' : 0.5677013087416922,
        'max_depth' : 4,
        'min_child_weight' : 2.4500998564395142,
        'objective' : 'reg:linear',
        'reg_alpha' : 0.028122426752245112,
        'reg_lambda' : 0.02914442678415895,
        'seed' : 42
    }
    mlflow.set_tag("model", "xgboost")
    mlflow.log_params(best_params)
    
    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )
    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)
    
    mlflow.xgboost.log_model(booster,artifact_path="modles_mlflow")

[0]	validation-rmse:6.50204
[1]	validation-rmse:5.81400
[2]	validation-rmse:5.65345
[3]	validation-rmse:5.60201
[4]	validation-rmse:5.57109
[5]	validation-rmse:5.56029


/home/codespace/anaconda3/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [06:43:30] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[6]	validation-rmse:5.55226
[7]	validation-rmse:5.54431
[8]	validation-rmse:5.53321
[9]	validation-rmse:5.52937
[10]	validation-rmse:5.52428
[11]	validation-rmse:5.51853
[12]	validation-rmse:5.51263
[13]	validation-rmse:5.50889
[14]	validation-rmse:5.50694
[15]	validation-rmse:5.50405
[16]	validation-rmse:5.50023
[17]	validation-rmse:5.49328
[18]	validation-rmse:5.48924
[19]	validation-rmse:5.48700
[20]	validation-rmse:5.48035
[21]	validation-rmse:5.47657
[22]	validation-rmse:5.47315
[23]	validation-rmse:5.47044
[24]	validation-rmse:5.46823
[25]	validation-rmse:5.46603
[26]	validation-rmse:5.46395
[27]	validation-rmse:5.46275
[28]	validation-rmse:5.45992
[29]	validation-rmse:5.45885
[30]	validation-rmse:5.45667
[31]	validation-rmse:5.45554
[32]	validation-rmse:5.45375
[33]	validation-rmse:5.45304
[34]	validation-rmse:5.45065
[35]	validation-rmse:5.44906
[36]	validation-rmse:5.44688
[37]	validation-rmse:5.44579
[38]	validation-rmse:5.43652
[39]	validation-rmse:5.43140
[40]	validation-rm

[283]	validation-rmse:5.20622
[284]	validation-rmse:5.20473
[285]	validation-rmse:5.20435
[286]	validation-rmse:5.20406
[287]	validation-rmse:5.20369
[288]	validation-rmse:5.20351
[289]	validation-rmse:5.20344
[290]	validation-rmse:5.20286
[291]	validation-rmse:5.20286
[292]	validation-rmse:5.20227
[293]	validation-rmse:5.20229
[294]	validation-rmse:5.20210
[295]	validation-rmse:5.20153
[296]	validation-rmse:5.20147
[297]	validation-rmse:5.20081
[298]	validation-rmse:5.20045
[299]	validation-rmse:5.20110
[300]	validation-rmse:5.20103
[301]	validation-rmse:5.20093
[302]	validation-rmse:5.20087
[303]	validation-rmse:5.20073
[304]	validation-rmse:5.20044
[305]	validation-rmse:5.20103
[306]	validation-rmse:5.20151
[307]	validation-rmse:5.20117
[308]	validation-rmse:5.20103
[309]	validation-rmse:5.20081
[310]	validation-rmse:5.20061
[311]	validation-rmse:5.20083
[312]	validation-rmse:5.19704
[313]	validation-rmse:5.19675
[314]	validation-rmse:5.19555
[315]	validation-rmse:5.19365
[316]	vali

[557]	validation-rmse:5.13255
[558]	validation-rmse:5.13043
[559]	validation-rmse:5.12866
[560]	validation-rmse:5.12767
[561]	validation-rmse:5.12797
[562]	validation-rmse:5.12776
[563]	validation-rmse:5.12761
[564]	validation-rmse:5.12746
[565]	validation-rmse:5.12738
[566]	validation-rmse:5.12708
[567]	validation-rmse:5.12600
[568]	validation-rmse:5.12596
[569]	validation-rmse:5.12597
[570]	validation-rmse:5.12618
[571]	validation-rmse:5.12602
[572]	validation-rmse:5.12597
[573]	validation-rmse:5.12584
[574]	validation-rmse:5.12593
[575]	validation-rmse:5.12593
[576]	validation-rmse:5.12511
[577]	validation-rmse:5.12520
[578]	validation-rmse:5.12525
[579]	validation-rmse:5.12529
[580]	validation-rmse:5.12523
[581]	validation-rmse:5.12536
[582]	validation-rmse:5.12226
[583]	validation-rmse:5.12201
[584]	validation-rmse:5.12169
[585]	validation-rmse:5.12115
[586]	validation-rmse:5.11900
[587]	validation-rmse:5.11833
[588]	validation-rmse:5.11826
[589]	validation-rmse:5.11861
[590]	vali

/home/codespace/anaconda3/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [06:43:45] WARNING: /workspace/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)
/home/codespace/anaconda3/lib/python3.9/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [22]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

mlflow.sklearn.autolog()

for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

    with mlflow.start_run():

        mlflow.log_param("train-data-path", "./data/green_tripdata_2024-01.csv")
        mlflow.log_param("valid-data-path", "./data/green_tripdata_2024-02.csv")
        mlflow.log_artifact("models/preprocessor.b", artifact_path="prerocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

2024/07/13 21:31:33 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
2024/07/13 21:34:59 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpjgfw0s2g/model/model.pkl, flavor: sklearn). Fall back to return ['scikit-learn==1.0.2', 'cloudpickle==2.0.0']. Set logging level to DEBUG to see the full traceback. 
2024/07/13 21:34:59 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/codespace/anaconda3/lib/python3.9/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils."
2024/07/13 21:35:04 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
2024/07/13 21:35:11 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarr

In [23]:
# Predict on a Pandas DataFrame:

import mlflow
logged_model = 'runs:/bbb2596bf535457da2c053c82e10632b/model'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

In [29]:
pred = loaded_model.predict(X_val)

In [30]:
pred[0:10]

array([20.1899    , 23.74216667,  9.8468752 ,  8.63886396,  2.59794444,
        5.08358333,  8.63886396, 18.01061111, 19.42138889,  6.41731907])

In [31]:
y_val[0:10]

array([ 8.45      , 27.86666667,  5.05      , 10.18333333,  3.78333333,
        3.35      ,  6.45      , 10.7       , 21.13333333,  3.95      ])